# 파이토치로 로지스틱회귀 (이진분류) 구현 

In [2]:
import pandas as pd
df = pd.read_excel("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/MouseProtein/mouse_protein_X.xls", header=None)
df_label = pd.read_excel("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/MouseProtein/mouse_protein_label.xls", header=None)

In [3]:
# 샘플당 69개의 특성(feature)을 가진 데이터
df.head()

,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,0.50364,0.74719,0.43018,2.8163,5.9902,0.21883,0.17757,2.3737,0.23222,1.7509,...,0.14276,0.43096,0.24754,1.6033,2.0149,0.10823,1.04500,0.83156,0.18885,1.6757
1,0.51462,0.68906,0.41177,2.7895,5.6850,0.21164,0.17282,2.2921,0.22697,1.5964,...,0.14204,0.45716,0.25763,1.6717,2.0046,0.10975,1.00990,0.84927,0.20040,1.7436
2,0.50918,0.73025,0.41831,2.6872,5.6221,0.20901,0.17572,2.2833,0.23025,1.5613,...,0.14244,0.51047,0.25534,1.6635,2.0168,0.10820,0.99685,0.84671,0.19368,1.9264
3,0.44211,0.61708,0.35863,2.4669,4.9795,0.22289,0.17646,2.1523,0.20700,1.5951,...,0.14507,0.43100,0.25110,1.4846,1.9572,0.11988,0.99022,0.83328,0.19211,1.7006
4,0.43494,0.61743,0.35880,2.3658,4.7187,0.21311,0.17363,2.1340,0.19216,1.5042,...,0.14087,0.48123,0.25177,1.5348,2.0091,0.11952,0.99777,0.87867,0.20560,1.8397


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [5]:
torch.manual_seed(1)

In [10]:
X_train = torch.FloatTensor(df.values)

df_label[0] = [1 if i==1 else 0 for i in df_label[0]]
y_train = torch.FloatTensor(df_label.values)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                                    stratify=y_train,
                                                    test_size=0.3,
                                                    random_state=42)

In [12]:
# 파이토치 클래스 생성
class BinaryClassifier(nn.Module):
  def __init__(self, input_dim):
    super().__init__()
    self.linear = nn.Linear(input_dim, 1) # 이진분류니까 output_dim = 1
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.linear(x))

In [38]:
# 파이토치 모델 선언
input_dim = X_train.shape[1]
model = BinaryClassifier(input_dim)

In [49]:
# 훈련 파라미터 선언
epochs = 10000
optimizer = optim.SGD(params=model.parameters(),
                      lr=0.1)

In [50]:
for epoch in range(epochs):
  hypothesis = model(X_train)
  loss = F.binary_cross_entropy(hypothesis, y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [56]:
pred = model(X_test)
pred = [1 if i>= 0.5 else 0 for i in pred]

In [57]:
accuracy = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

In [58]:
print(f'accuracy: {accuracy:.3f}')
print(f'recall: {recall:.3f}')
print(f'precision: {precision:.3f}')
print(f'f1: {f1:.3f}')

accuracy: 0.957
recall: 0.971
precision: 0.949
f1: 0.960
